In [2]:
import pandas as pd

# === 入力パス ===
CAT_CSV      = "csv_exports/category.csv"
FC_CSV       = "csv_exports/film_category.csv"
FILM_CSV     = "csv_exports/film.csv"
INV_CSV      = "csv_exports/inventory.csv"
RENT_CSV     = "csv_exports/rental.csv"
CUS_CSV      = "csv_exports/customer.csv"
ADDR_CSV     = "csv_exports/address.csv"
CITY_CSV     = "csv_exports/city.csv"
COUNTRY_CSV  = "csv_exports/country.csv"

def make_country_category_counts():
    # 必要列だけ読む（last_update等は読まない）
    cat   = pd.read_csv(CAT_CSV, usecols=["category_id","name"])
    fc    = pd.read_csv(FC_CSV,  usecols=["film_id","category_id"])
    inv   = pd.read_csv(INV_CSV, usecols=["inventory_id","film_id"])
    rent  = pd.read_csv(RENT_CSV, usecols=["rental_id","inventory_id","customer_id"], parse_dates=["rental_id"], dayfirst=False)  # rental_idは数値列でもOK
    # ↑ parse_datesは不要なら削除してOK（安全のため残しても害はありません）

    cus   = pd.read_csv(CUS_CSV,  usecols=["customer_id","address_id"])
    addr  = pd.read_csv(ADDR_CSV, usecols=["address_id","city_id"])
    city  = pd.read_csv(CITY_CSV, usecols=["city_id","country_id","city"])
    country = pd.read_csv(COUNTRY_CSV, usecols=["country_id","country"])

    # 顧客→住所→都市→国（必要列のみ）
    cus_country = (cus.merge(addr,  on="address_id", how="left")
                       .merge(city, on="city_id",   how="left")
                       .merge(country, on="country_id", how="left"))
    # rental -> customer -> country
    r_cus = rent.merge(cus_country[["customer_id","country"]], on="customer_id", how="left")

    # rental -> inventory -> film -> film_category -> category名
    df = (r_cus.merge(inv, on="inventory_id", how="left")
              .merge(fc,  on="film_id",      how="left")
              .merge(cat, on="category_id",  how="left"))

    # ピボット（件数）
    pt = pd.pivot_table(df, index="country", columns="name",
                        values="rental_id", aggfunc="count", fill_value=0)

    # 国名・カテゴリ名の昇順で整形（任意）
    pt = pt.sort_index().sort_index(axis=1)
    return pt

if __name__ == "__main__":
    pt_country = make_country_category_counts()
    out_csv = "country_category_counts.csv"
    pt_country.to_csv(out_csv, encoding="utf-8-sig")
    print(f"出力: {out_csv}")


出力: country_category_counts.csv


C:\Users\flare\AppData\Local\Temp\ipykernel_26320\375473170.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  rent  = pd.read_csv(RENT_CSV, usecols=["rental_id","inventory_id","customer_id"], parse_dates=["rental_id"], dayfirst=False)  # rental_idは数値列でもOK
